### Manpower Planning data

#### 预计未来所需要的人数
| <i></i> | Unskilled | Semi-skilled | Skilled |
| --- | --- | --- | --- |
| Current Strength | 2000 | 1500 | 1000 |
| Year 1 | 1000 | 1400 | 1000 |
| Year 2 | 500 | 2000 | 1500 |
| Year 3 | 0 | 2500 | 2000 |

#### 新员工的离职率
| <i></i> | Unskilled (%)| Semi-skilled (%) | Skilled (%) |
| --- | --- | --- | --- |
| $< 1$ year of service | 25 | 20 | 10 |
| $\geq 1$ year of service | 10 | 5 | 5 |

#### 可招聘的最大人数限制
| <i></i> | Unskilled | Semi-skilled | Skilled |
| --- | --- | --- | --- |
| Year 1 | 500 | 800 | 500 |
| Year 2 | 500 | 800 | 500 |
| Year 3 | 500 | 800 | 500 |

#### 每年可转岗限制(人数(一人成本))
| <i></i> | Unskilled | Semi-skilled | Skilled |
| --- | --- | --- | --- |
| Unskilled | 0 | 200(400/人) | - |
| Semi-skilled | 0 | 0 | <=0.25*Skilled(500/人) |
| Skilled | 0 | 0 | 0 |

    50%的降级员工有可能会离开公司；

#### 开除员工成本(成本/人)
| <i></i> | Unskilled | Semi-skilled | Skilled |
| --- | --- | --- | --- |
|开除成本| 200 | 500 | 500 |

#### 超员额外支出成本(成本/人)
| <i></i> | Unskilled | Semi-skilled | Skilled |
| --- | --- | --- | --- |
|超员成本| 1500 | 2000 | 3000 |

    总超员人数<=150人

#### 短期员工的限制(成本/人/年)
| <i></i> | Unskilled | Semi-skilled | Skilled |
| --- | --- | --- | --- |
|带人成本| 500 | 400 | 400 |

    每个级别可带人的数<=50;
    短期员工的生产力是正式员工的一半;







In [1]:
data = dict()

In [1]:
years = [1, 2, 3]
skills = ['s1', 's2', 's3']
curr_workforce = {'s1': 2000, 's2': 1500, 's3': 1000}
demand = {
    (1, 's1'): 1000,
    (1, 's2'): 1400,
    (1, 's3'): 1000,
    (2, 's1'): 500,
    (2, 's2'): 2000,
    (2, 's3'): 1500,
    (3, 's1'): 0,
    (3, 's2'): 2500,
    (3, 's3'): 2000
}

max_hiring = {
    (1, 's1'): 500,
    (1, 's2'): 800,
    (1, 's3'): 500,
    (2, 's1'): 500,
    (2, 's2'): 800,
    (2, 's3'): 500,
    (3, 's1'): 500,
    (3, 's2'): 800,
    (3, 's3'): 500
}
max_overmanning = 150
max_parttime = 50
parttime_cap = 0.50
max_train_unskilled = 200
max_train_semiskilled = 0.25

rookie_attrition = {'s1': 0.25, 's2': 0.20, 's3': 0.10}
veteran_attrition = {'s1': 0.10, 's2': 0.05, 's3': 0.05}
demoted_attrition = 0.50

training_cost = {'s1': 400, 's2': 500}
layoff_cost = {'s1': 200, 's2': 500, 's3': 500}
parttime_cost = {'s1': 500, 's2': 400, 's3': 400}
overmanning_cost = {'s1': 1500, 's2': 2000, 's3': 3000}



In [2]:
import pyscipopt as scip

In [13]:
model = scip.Model()

In [14]:
### 定义变量
employed = dict()
recruited = dict()
retrained = dict()
redundant = dict()
shorttime = dict()
overmanning = dict()

for year in years:
    for skill in skills:
        employed[year,skill] = model.addVar(name=f"employed_{year}_{skill}",vtype="C")
        recruited[year,skill] = model.addVar(name=f"recruited_{year}_{skill}",vtype="C",ub=max_hiring[year,skill])
        redundant[year,skill] = model.addVar(name=f"redundant_{year}_{skill}",vtype="C")
        shorttime[year,skill] = model.addVar(name=f"shorttime_{year}_{skill}",vtype="C",ub=max_parttime)
        overmanning[year,skill] = model.addVar(name=f"overmanning_{year}_{skill}",vtype="C")
        for other_skill in skills:
            retrained[year,skill,other_skill] = model.addVar(name=f"retrained_{year}_{skill}_{other_skill}",vtype="I")



In [15]:
### 定义约束条件
Balance = dict()
for year in years:
    for skill in skills:
        if year == 1:
            pre_employed = curr_workforce[skill]
        else:
            pre_employed = employed[year-1,skill]
        pre_employed = (1-veteran_attrition[skill])*pre_employed
        cur_recruited = (1-rookie_attrition[skill])*recruited[year, skill]
        cur_retrained = scip.quicksum((1- veteran_attrition[skill])*retrained[year,skill1,skill] - retrained[year,skill,skill1]
                                     for skill1 in skills if skill1 < skill)
        cur_retrained += scip.quicksum((1- demoted_attrition)*retrained[year,skill1,skill] - retrained[year,skill,skill1]
                                     for skill1 in skills if skill1 > skill)
        cur_redundant = redundant[year,skill]      
        Balance[year,skill] = model.addCons(employed[year,skill] == pre_employed + cur_recruited + cur_retrained - cur_redundant)



for year in years:        
    model.addCons(retrained[year,"s1","s2"] <= max_train_unskilled)
    model.addCons(retrained[year,"s1","s3"] == 0)
    model.addCons(retrained[year,"s2","s3"] <= max_train_semiskilled * employed[year,"s3"])

for year in years:
    model.addCons(sum(overmanning[year,skill] for skill in skills) <= max_overmanning)


for year in years:
    for skill in skills:
        model.addCons(employed[year,skill] == demand[year,skill]
                      + overmanning[year,skill] + parttime_cap*shorttime[year,skill])

    
        
        

In [16]:
obj = scip.quicksum(redundant[year,skill] for year in years for skill in skills)
model.setObjective(obj, "minimize")


In [17]:

model.optimize()

In [18]:
model.getStatus()

'optimal'

In [19]:
model.getObjVal()

841.8199999999997

In [20]:
len(model.getVars())

72

In [26]:
#### 分析结果
import pandas as pd

In [27]:
rows = years.copy()
columns = skills.copy()
hire_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in recruited.keys():
    if (abs(model.getVal(recruited[year, level])) > 1e-6):
        hire_plan.loc[year, level] = np.round(model.getVal(recruited[year, level]), 1)
hire_plan

,s1,s2,s3
1,0.0,0.0,0.0
2,0.0,799.2,398.7
3,0.0,800.0,371.8


In [28]:
rows = years.copy()
columns = ['{0} to {1}'.format(level1, level2) for level1 in skills for level2 in skills if level1 != level2]
train_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level1, level2 in retrained.keys():
    col = '{0} to {1}'.format(level1, level2)
    if (abs(model.getVal(retrained[year, level1, level2])) > 1e-6):
        train_plan.loc[year, col] = np.round(model.getVal(retrained[year, level1, level2]), 1)
train_plan

,s1 to s2,s1 to s3,s2 to s1,s2 to s3,s3 to s1,s3 to s2
1,200.0,0.0,0.0,256.0,0.0,168.0
2,200.0,0.0,0.0,176.0,0.0,0.0
3,200.0,0.0,0.0,253.0,0.0,0.0


In [29]:
rows = years.copy()
columns = skills.copy()
layoff_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in redundant.keys():
    if (abs(model.getVal(redundant[year, level])) > 1e-6):
        layoff_plan.loc[year, level] = np.round(model.getVal(redundant[year, level]), 1)
layoff_plan

,s1,s2,s3
1,443.2,0.0,0.0
2,166.1,0.0,0.0
3,232.5,0.0,0.0


In [30]:
rows = years.copy()
columns = skills.copy()
parttime_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in shorttime.keys():
    if (abs(model.getVal(shorttime[year, level])) > 1e-6):
        parttime_plan.loc[year, level] = np.round(model.getVal(shorttime[year, level]), 1)
parttime_plan

,s1,s2,s3
1,50.0,50.0,50.0
2,50.0,48.4,0.0
3,50.0,0.0,0.0


In [31]:
rows = years.copy()
columns = skills.copy()
excess_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for year, level in overmanning.keys():
    if (abs(model.getVal(overmanning[year, level])) > 1e-6):
        excess_plan.loc[year, level] = np.round(model.getVal(overmanning[year, level]), 1)
excess_plan

,s1,s2,s3
1,131.8,18.0,0.2
2,150.0,0.0,0.0
3,150.0,0.0,0.0
